## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import pprint
import json

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Bluff,NZ,-46.6000,168.3333,44.80,84,100,23.20,heavy intensity rain
1,1,Jamestown,US,42.0970,-79.2353,71.10,68,90,5.99,mist
2,2,Hakkari,TR,37.5833,44.1667,55.65,45,0,6.17,clear sky
3,3,Bredasdorp,ZA,-34.5322,20.0403,50.29,87,13,0.00,few clouds
4,4,Atuona,PF,-9.8000,-139.0333,77.56,74,15,17.78,few clouds
5,5,Hermanus,ZA,-34.4187,19.2345,54.90,71,37,7.11,scattered clouds
6,6,Cape Town,ZA,-33.9258,18.4232,57.27,55,20,12.66,few clouds
7,7,Nikolskoye,RU,59.7035,30.7861,50.40,85,0,6.71,clear sky
8,8,Bambous Virieux,MU,-20.3428,57.7575,72.21,88,75,13.80,broken clouds
9,9,Sao Luiz Gonzaga,BR,-28.4083,-54.9608,65.97,91,99,12.35,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 60


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
2,2,Hakkari,TR,37.5833,44.1667,55.65,45,0,6.17,clear sky
3,3,Bredasdorp,ZA,-34.5322,20.0403,50.29,87,13,0.00,few clouds
5,5,Hermanus,ZA,-34.4187,19.2345,54.90,71,37,7.11,scattered clouds
6,6,Cape Town,ZA,-33.9258,18.4232,57.27,55,20,12.66,few clouds
7,7,Nikolskoye,RU,59.7035,30.7861,50.40,85,0,6.71,clear sky
10,10,Port Lincoln,AU,-34.7333,135.8667,52.25,67,76,25.84,broken clouds
12,12,New Norfolk,AU,-42.7826,147.0587,53.74,82,49,1.28,scattered clouds
15,15,Constitucion,CL,-35.3333,-72.4167,59.11,53,0,19.62,clear sky
32,32,Port Macquarie,AU,-31.4333,152.9167,57.16,67,100,6.53,overcast clouds
34,34,Walvis Bay,NaN,-22.9575,14.5053,56.35,76,0,4.61,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        103
City           103
Country        101
Lat            103
Lng            103
Max Temp       103
Humidity       103
Cloudiness     103
Wind Speed     103
Description    103
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID        101
City           101
Country        101
Lat            101
Lng            101
Max Temp       101
Humidity       101
Cloudiness     101
Wind Speed     101
Description    101
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Hakkari,TR,55.65,clear sky,37.5833,44.1667,
3,Bredasdorp,ZA,50.29,few clouds,-34.5322,20.0403,
5,Hermanus,ZA,54.90,scattered clouds,-34.4187,19.2345,
6,Cape Town,ZA,57.27,few clouds,-33.9258,18.4232,
7,Nikolskoye,RU,50.40,clear sky,59.7035,30.7861,
10,Port Lincoln,AU,52.25,broken clouds,-34.7333,135.8667,
12,New Norfolk,AU,53.74,scattered clouds,-42.7826,147.0587,
15,Constitucion,CL,59.11,clear sky,-35.3333,-72.4167,
32,Port Macquarie,AU,57.16,overcast clouds,-31.4333,152.9167,
38,Dali,CN,59.81,scattered clouds,25.7000,100.1833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
     # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found in", row['City'], ".. skipping.")
        hotel_df.loc[index, "Hotel Name"] = "NA"

Hotel not found in Hakkari .. skipping.
Hotel not found in Yandyki .. skipping.
Hotel not found in Tolstoy-Yurt .. skipping.
Hotel not found in Sinop .. skipping.
Hotel not found in Anuchino .. skipping.
Hotel not found in Norashen .. skipping.
Hotel not found in Zavetnoye .. skipping.
Hotel not found in Irgakly .. skipping.
Hotel not found in Krasnorechenskiy .. skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df=hotel_df[hotel_df["Hotel Name"] != "NA"]
pd.options.display.float_format = "{:,.2f}".format
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Bredasdorp,ZA,50.29,few clouds,-34.53,20.04,Bredasdorp Country Manor
5,Hermanus,ZA,54.90,scattered clouds,-34.42,19.23,Misty Waves Boutique Hotel
6,Cape Town,ZA,57.27,few clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
7,Nikolskoye,RU,50.40,clear sky,59.70,30.79,Tourist House - Sablino
10,Port Lincoln,AU,52.25,broken clouds,-34.73,135.87,Port Lincoln Hotel
...,...,...,...,...,...,...,...
633,Visnes,NO,57.58,scattered clouds,59.36,5.22,Park Inn Haugesund Airport
645,Stornoway,GB,53.92,broken clouds,58.21,-6.39,Caladh Inn
663,Port Augusta,AU,55.54,clear sky,-32.50,137.77,Majestic Oasis Apartments
679,Burnie,AU,53.92,light rain,-41.07,145.92,Beachfront Voyager Motor Inn


In [17]:
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
#8a Export the City_Data into a CSV.
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd> and <dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)



# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))